# Análise Exploratória do dados do Enem 2016

A seguinte análise serve para conhecer melhor os dados disponíveis, apresentar alguma técnicas básicas de análise utilizando pandas e responder a algumas perguntas sobre os dados:
* número total de alunos do Enem 2016:

* relação entre desempenho e 
    - renda dos pais
    - gênero
    - raça
    - tipo de escola, pública ou privada
    - estado de origem
    - etc.

* reproduzir a análise e os resultados do excelente exemplo de jornalismo de dados realizado pelo jornal Estadão: 
http://infograficos.estadao.com.br/educacao/enem/desigualdades-de-genero-e-raca/


Dados do Enem publicados pelo Inep: http://portal.inep.gov.br/microdados

Dicionário de dados disponível junto com os dados: http://download.inep.gov.br/microdados/microdados_enem2016.zip (1 GB, compressado)

Aproveitei código de: [Gustavo Bonesso](https://www.kaggle.com/gbonesso)



In [16]:
# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import gc

#%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))

In [ ]:
# fonte: ../input/microdados_enem_2016_coma.csv
# Lê só as 5 primeiras linhas
df = pd.read_csv("../input/microdados_enem_2016_coma.csv", nrows=5, encoding='iso-8859-1')

In [19]:
# Mostra o nome das colunas
print(list(df.columns))
len(df.columns)

A semântica das variáveis representadas nas colunas do dataset está descrita no dicionário de dados.

Assim, podemos importar seletivamente apenas as colunas que sejam necessárias para a análise.

Exemplo:


In [ ]:
colunas = ['SG_UF_RESIDENCIA', 'Q006', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 
          'NU_NOTA_MT', 'NU_NOTA_REDACAO']
df = pd.read_csv('../input/microdados_enem_2016_coma.csv', encoding='latin-1', 
                 sep=',', usecols=colunas)
len(df)

In [ ]:
# verificação de dados faltantes
total = df.isnull().sum().sort_values(ascending = False)
percentual = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
faltantes  = pd.concat([total, percentual], axis=1, keys=['Total', 'Percentual'])
faltantes

In [10]:
df.info()

In [ ]:
# Cria nota média
df['NOTA_MEDIA'] = (
    df['NU_NOTA_CN'] + df['NU_NOTA_CH'] + df['NU_NOTA_LC'] + df['NU_NOTA_MT'] + 
    df['NU_NOTA_REDACAO']) / 5.0

df.head()

In [12]:
# Elimina as linhas com nota média NaN, que provavelmente corresponde a alunos ausentes a alguma das provas
df.dropna(inplace=True)
print(len(df))
df.head()

In [ ]:
#2. --> Proporcionalmente à população de alunos do estado, qual o estado com melhor desempenho no ENEM?

# Selecionar os top 10 das médias de alunos

df.NOTA_MEDIA.qualtile(q=0.9)
nota_corte=df.NOTA_MEDIA.quantile(q=0.9)
nota_corte

In [ ]:
df.loc[df.NOTA_MEDIA>=nota_corte][["SG_UF_RESIDENCIA]].groupby

#### Sumarização simples

In [13]:
# só numéricos
df[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'NOTA_MEDIA']].describe()

In [14]:
# Dados categóricos
df.describe(include="O")

In [ ]:
# Pessoas com nota máxima em redação:
len(df[df.NU_NOTA_REDACAO == 1000])

In [ ]:
# Histograma das médias
df.NOTA_MEDIA.hist()

In [ ]:
# Ou mais elegante
sns.distplot(df.NOTA_MEDIA)

In [ ]:
# distribuição dos estudantes por estado
df.SG_UF_RESIDENCIA.value_counts()

In [ ]:
# De forma gráfica:
df.SG_UF_RESIDENCIA.value_counts().plot.bar(color='blue')

### Nota média por patamar de renda

A renda familiar é codificada na Q006, por fatia de renda, de 0 a acima de R$ 17.600, nas letras A a Q

In [ ]:
import string
# 17 primeiras letras maiúsculas
ordem_renda = list(string.ascii_uppercase[:17])
print(ordem_renda)
#Q006_order=["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q"]

In [ ]:
# distribuição do número de alunos por nível de renda
df.Q006.value_counts().plot.bar(color='blue')

In [ ]:
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)

sns.violinplot(
    x="Q006", 
    y="NOTA_MEDIA", 
    data=df, # caso queira recortar por estado: df[(df.SG_UF_RESIDENCIA == {código de estado})]
    order=ordem_renda
    )
plt.title("Distribuição de notas por patamar de renda familiar")
plt.show()

#### Distribuição de notas por estado da federação

In [ ]:
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)

sns.violinplot(
    x="SG_UF_RESIDENCIA",
    y="NOTA_MEDIA",
    data=df, # caso queira recortar por patamar de renda: df[(df.Q006 == {nível de renda, letra de A a Q})]
    )
plt.title("Distribuição de notas por estado")
plt.show()

## Reproduzir análise do Estadão
http://infograficos.estadao.com.br/educacao/enem/desigualdades-de-genero-e-raca/

O Estadão realiza diversas comparações entre a população geral dos alunos do Enem e o sub-grupo dos 1000 alunos que obtiveram as melhores notas. Esse recorte permite contrastar as características socioeconomicas e educacionais desses dois grupos.

Para refazer as análises do Estadão é necessário carregar as variáveis sexo, raça, escolaridade dos pais, cidade de residência e tipo de escola (pública/privada).

Da leitura do dicionário de dados, vê-se que essas são as variáveis: 
'TP_SEXO', 'TP_COR_RACA', 'Q001', 'Q002', 'NO_MUNICIPIO_RESIDENCIA', 'TP_ESCOLA'

Obs: 'Q001': escolaridade do pai, 'Q002': escolaridade da mãe

In [ ]:
# Liberar a memória utilizada pelo atual dataset:
del df
df = ""
gc.collect()

In [ ]:
colunas = ['SG_UF_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA','TP_SEXO', 'TP_COR_RACA', 'TP_ESCOLA', 'Q001', 'Q002', 'Q006', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 
          'NU_NOTA_MT', 'NU_NOTA_REDACAO']
df = pd.read_csv('../input/microdados_enem_2016_coma.csv', encoding='latin-1', 
                 sep=',', usecols=colunas)
len(df)

In [ ]:
# Novamente, eliminar as linhas com nota faltante
df.dropna(inplace=True)

# Cria nota média
df['NOTA_MEDIA'] = (
    df['NU_NOTA_CN'] + df['NU_NOTA_CH'] + df['NU_NOTA_LC'] + df['NU_NOTA_MT'] + 
    df['NU_NOTA_REDACAO']) / 5.0

print(len(df))

In [ ]:
# Selecionar os 1000 alunos com as melhores notas

# nota do milésimo aluno
top_notas = df.NOTA_MEDIA.nlargest(1000)

# distribuição das 1000 melhores médias
sns.distplot(top_notas)

In [ ]:
# menor das 1000 notas
nota_corte = top_notas.min()
nota_corte

In [ ]:
# Seleção dos 1000 alunos de maior média
top1000 = df[df.NOTA_MEDIA >= nota_corte]
len(top1000)

In [ ]:
# Comparações de proporção de gênero entre top1000 e população
df.TP_SEXO.value_counts()/len(df)*100

In [ ]:
top1000.TP_SEXO.value_counts()/len(top1000)*100

Aparente divergência de resultado com o anunciado pelo Estadão de que "mais de 70% dos estudantes que tiraram as mil maiores notas no Exame Nacional do Ensino Médio (Enem) são meninos."

Talvez o jornal tenha selecionado as "mil maiores notas" de forma diferente. Aqui, foram selecionadas as mil maiores médias, com igual ponderação de todas as provas.

In [ ]:
# Correlação entre as notas?
colunas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
# colunas = ['TP_SEXO', 'TP_COR_RACA', 'TP_ESCOLA', 'Q001', 'Q002', 'Q006', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

#sns.set()
#sns.pairplot(df[colunas], size = 2.5)
#plt.show()

In [ ]:
# Como a quantidade de dados é muito elevada, os gráficos de correlação demoram muito para serem calculados e ficam saturados
# Uma solução sistemática é realizar uma subamostragem da população [aqui, um milésimo da população]
amostra = df.sample(frac=0.001, random_state=123)
len(amostra)

In [ ]:
sns.set()
sns.pairplot(amostra[colunas], size = 3.5, kind='reg', plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.005}})
# defino um alpha muito baixo devido ao grande número de pontos, para limitar o efeito de saturação
plt.show()

In [ ]:
4+2

*
**To be continued...*